In [ ]:
import pandas as pd
from typing import List
from logging import getLogger


In [53]:
logger = getLogger(__name__)

In [1]:
%pip install fastapi

Note: you may need to restart the kernel to use updated packages.


In [ ]:
product_df = pd.read_csv('../data/openfoodfacts.csv')  # twój DataFrame z produktami

In [73]:
test_df = product_df.sample(10000)

In [78]:
test_df.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,...,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
387723,2130270007350,http://world-en.openfoodfacts.org/product/2130...,kiliweb,1598803193,2020-08-30T15:59:53Z,1602780048,2020-10-15T16:40:48Z,teolemon,1.729977e+09,2024-10-26T21:07:56Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060287,8431444000233,http://world-en.openfoodfacts.org/product/8431...,openfoodfacts-contributors,1550154633,2019-02-14T14:30:33Z,1598388884,2020-08-25T20:54:44Z,thaialagata,1.729977e+09,2024-10-26T21:17:57Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78092,39978001511,http://world-en.openfoodfacts.org/product/0039...,usda-ndb-import,1489055395,2017-03-09T10:29:55Z,1564063010,2019-07-25T13:56:50Z,roboto-app,1.707498e+09,2024-02-09T16:51:41Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
656735,3760206162571,http://world-en.openfoodfacts.org/product/3760...,kiliweb,1658859365,2022-07-26T18:16:05Z,1664913655,2022-10-04T20:00:55Z,roboto-app,1.707742e+09,2024-02-12T12:39:16Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264494,667888267125,http://world-en.openfoodfacts.org/product/0667...,kiliweb,1603645591,2020-10-25T17:06:31Z,1678933509,2023-03-16T02:25:09Z,anonymous-s7co2zv64u,1.729932e+09,2024-10-26T08:39:12Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
